In [1]:
import cdsapi
import numpy as np
import xarray as xr
import matplotlib.pyplot as plt
import cfgrib
import cartopy.crs as ccrs
import cartopy.feature as cfeature
from datetime import datetime
import pandas as pd
import os
import geopandas as gpd
from shapely.geometry import Point
import cdsapi
import numpy as np
from itertools import chain, count
import asf_search as asf

In [2]:
workspace_directory = r'C:\Users\trygg\Documents\Master_project' #HOME
# workspace_directory = r'C:\Users\tryggvisi\Documents\my-awesome-masters-project' #WORK
os.chdir(workspace_directory)  # Change the current working directory to the specified workspace

Þetta script er til að ná í fleiri granules með því að nota era5 data.

Fyrstu kaflar eru til að prófa og testa era5

Main processing er til að útbúa lista af granules og exporta honum til að processa.

## ERA5 request

In [ ]:
#For testing
#For testing if meteoroligical data is useable
c = cdsapi.Client()

c.retrieve(
    "reanalysis-era5-single-levels",
    {
        "product_type": "reanalysis",
        'variable': [
                'sf'

        ],
        "format": "netcdf",
        "area": [
            67,
            15,
            65,
            17,
        ],
        "day": [
"01","02","03","04","05","06","07","08","09","10",
"11","12","13","14","15","16","17","18","19","20",
"21","22","23","24","25","26","27","28","29","30","31"

        ],
        "time": [
                '00:00','01:00','02:00',
                '03:00','04:00','05:00',
                '06:00','07:00','08:00',
                '09:00','10:00','11:00',
                '12:00','13:00','14:00',
                '15:00','16:00','17:00',
                '18:00','19:00','20:00',
                '21:00','22:00','23:00'
        ],
        "month": ["01","02","03","04"],
        "year": ["2017","2018","2019","2020","2021","2022",
            "2023"],

    },
    "download.nc",
)

## Total precipitation




In [ ]:
#total precip tp
# netcdf_file_path = r"Data\ERA5\ERA5_more_granules\adaptor.mars.internal-1711660184.5711513-21277-4-e2435d20-3cd5-428e-840c-5196f04e4764.nc" #með einum mánuði
# netcdf_file_path = r"Data\ERA5\ERA5_more_granules\adaptor.mars.internal-1711662622.906625-24049-10-26c12f28-1bd5-4782-a72a-b22d6fd96626.nc"# einn máuður, mörg ár
netcdf_file_path = r"Data\ERA5\ERA5_more_granules\adaptor.mars.internal-1711667538.722831-31100-8-5f4966dd-a0b8-4150-b18d-ebab4389031d.nc" #mörg ár, mörg mánuðir
ds_nc_tp = xr.open_dataset(netcdf_file_path)

#Snowfall sf
netcdf_file_path = r"Data\ERA5\ERA5_more_granules\adaptor.mars.internal-1711891504.5908513-27584-16-3e7ddec3-7f5d-45f1-8481-8111ed159f28.nc" #mörg á margir ´mauðir

ds_nc_sf = xr.open_dataset(netcdf_file_path)


In [ ]:
#
def dataset_resample(ds):
    ds_resample = ds.resample(time='1d').sum()
    return ds_resample

ds_resample_tp = dataset_resample(ds_nc_tp)
ds_resample_tp = ds_resample_tp*1000
ds_resample_tp_where = ds_resample_tp.where(ds_resample_tp < 5, drop=True)


plt.figure(figsize=(30, 12))
plt.axhline(10, color='black', lw=3)
ds_resample_tp.tp.sel( latitude=65.75, longitude=15, method='nearest').plot(color='blue', label='Total Precipitation')
ds_resample_tp_where.tp.sel( latitude=65.75, longitude=15, method='nearest').plot(color='red', label='Limit of 10mm/day')

# ds_resample_sf.sel(time='2021').sel(latitude=65.75, longitude=15, method='nearest').plot(color='red', label='Snowfall')
plt.legend()
plt.show()

In [ ]:
lat_max, lat_min = 65.75, 66.25  # Example latitude bounds
lon_min, lon_max = 15.25, 16.75  # Example longitude bounds
ds_resample_tp_clip = ds_resample_tp.sel(latitude=slice(lat_min, lat_max), longitude=slice(lon_min, lon_max)) 


In [ ]:
ds_resample_tp_clip.sel(time='2023-02').sel(latitude=65.75, longitude=15, method='nearest').plot()

# Temperature

In [ ]:
ds_grib.resample(time='1d').sum()
ds_sf = ds_grib.resample(time='1d').sum()

lat_max, lat_min = 65.75, 66.25  # Example latitude bounds
lon_min, lon_max = 15.25, 16.75  # Example longitude bounds
ds_sf_clip = ds_sf.sel(latitude=slice(lat_min, lat_max), longitude=slice(lon_min, lon_max)) 

ds_grib.sf.isel(step=0, time=1).plot()

In [ ]:
path_to_nc = r"Data\ERA5\ERA5_more_granules\adaptor.mars.internal-1711747862.0604734-12346-11-be9af1ca-92fb-4398-8d81-62ad7a995866.nc" #öll ár og mánuðir, allt svæði og allir tímar
ds_grib = xr.open_dataset(path_to_nc)


In [ ]:
for v in ds_grib:
    print("{}, {}, {}".format(v, ds_grib[v].attrs["long_name"], ds_grib[v].attrs["units"]))

In [ ]:
ds_resampled = ds_grib.resample(time='1d').mean()
ds_resampled_t2m = ds_resampled.t2m-273.15
ds_resmapled_skt = ds_resampled.skt-273.15

lat_max, lat_min = 65.75, 66.25  # Example latitude bounds
lon_min, lon_max = 15.25, 16.75  # Example longitude bounds
ds_resampled_t2m = ds_resampled_t2m.sel(latitude=slice(lat_min, lat_max), longitude=slice(lon_min, lon_max)) 
ds_resampled_skt = ds_resmapled_skt.sel(latitude=slice(lat_min, lat_max), longitude=slice(lon_min, lon_max)) 

In [ ]:

#remove all values less than 0
ds_resampled_t2m = ds_resampled_t2m.where(ds_resampled_t2m < 0, drop=True)
ds_resampled_skt = ds_resampled_skt.where(ds_resampled_skt < 0, drop=True)

In [ ]:
#add a straight line at value 0 and across the x axis
plt.figure(figsize=(12, 8))     
plt.axhline(0, color='red', lw=3)

ds_resampled_t2m.sel( latitude=67, longitude=16, method='nearest').plot(color='blue', lw=.7)
ds_resampled_skt.sel( latitude=67, longitude=16, method='nearest').plot(color='green', lw=.7);
plt.show()

In [ ]:


#add the polygon from study_area_path to the plot
study_area_path = r"C:\Users\trygg\Documents\Master_project\Data\Vindefjallen_data\outputs\shapefiles\study_area_4326.shp"
study_area = gpd.read_file(study_area_path)
fig, ax = plt.subplots(1,1, figsize=(15, 6))
# gdf_plus_points.plot(column='snow_depth', cmap='viridis', legend=True, s=100, ax=ax)
clipped_ds.round(1).isel(time=0).plot(cmap='coolwarm')
study_area.boundary.plot(ax=ax, color='black')
plt.show()  # This line will display the plot


# Main processing

Import netcdf for precip and temp

- resample dataset to 1 day.
- Clip to study area
- Transform the data
- Filter out over certain threshold
- Loops through the dataset and determines if each date has enough valid pixels. Then averages the study area, and outputs a dataframe with date and mean values


In [3]:
#import the netcdf files
#total precipitation tp
netcdf_file_path = r"Data\ERA5\ERA5_more_granules\adaptor.mars.internal-1711667538.722831-31100-8-5f4966dd-a0b8-4150-b18d-ebab4389031d.nc" #mörg ár, mörg mánuðir
ds_nc_tp = xr.open_dataset(netcdf_file_path)

#temperature at 2m t2m
path_to_nc = r"Data\ERA5\ERA5_more_granules\adaptor.mars.internal-1711747862.0604734-12346-11-be9af1ca-92fb-4398-8d81-62ad7a995866.nc" #öll ár og mánuðir, allt svæði og allir tímar
ds_nc_t2m = xr.open_dataset(path_to_nc)


In [4]:
#resample the dataset to daily values
def dataset_resample(ds, type):
    #resample the dataset to daily values
    if type == 'sum':
        ds_resample = ds.resample(time='1d').sum()
    elif type == 'mean':
        ds_resample = ds.resample(time='1d').mean()
    return ds_resample

def dataset_clip(ds):
    #clip to study area
    lat_max, lat_min = 65.75, 66.25  # Example latitude bounds
    lon_min, lon_max = 15.25, 16.75  # Example longitude bounds
    ds_resample_clip = ds.sel(latitude=slice(lat_min, lat_max), longitude=slice(lon_min, lon_max)) 
    return ds_resample_clip 

In [5]:
# Resample and clip tp dataset
ds_resample_tp = dataset_resample(ds_nc_tp, 'sum')
ds_resample_tp_clip = dataset_clip(ds_resample_tp)
ds_resample_tp_clip = ds_resample_tp_clip * 1000
ds_resample_tp_clip_where = ds_resample_tp_clip.where(ds_resample_tp_clip <= 0.5, drop=True) #filter any values below 0.5 mm
ds_resample_tp_clip_where = ds_resample_tp_clip_where.tp

# Resample and clip t2m dataset
ds_resample_t2m = dataset_resample(ds_nc_t2m, 'mean')
ds_resample_t2m_clip = dataset_clip(ds_resample_t2m)
ds_resample_t2m_clip = ds_resample_t2m_clip - 273.15
ds_resample_t2m_clip_where = ds_resample_t2m_clip.where(ds_resample_t2m_clip <= -1, drop=True) #filter any values above 2mm/day
ds_resample_t2m_clip_where = ds_resample_t2m_clip_where.t2m

In [6]:
'''
This function takes a dataset and a variable type as input and returns a dataframe with the date, mean value and type of the variable.
The function loops through the dataset and extracts the mean value of the variable for each time step. 
It then extracts the time and mean value and adds it to a dictionary.
The dictionary is then converted to a dataframe and returned.
This function is applied to both precipitation and temperature datasets.
Finally the two dataframes are merged into one dataframe, and use to find granules
'''

def process_datasets(ds, var_type):
    teljari = 0
    dictionary_date_mean = {}

    for i in range(len(ds)):
        # all_number =  ds.isel(time=i).values
        mean_number = ds.isel(time=i).mean().values.round(1)
        time = ds.isel(time=i).time.values.astype(str)[:-19]
        
        #find the length of the array, and by extension finding the number of NaN values
        all_values = []
        # nested for loop to extract the 2D array values
        for value in ds.isel(time=i).values: # first dimension 
            # print(i)
            for j in value: #second dimension
                
                if not np.isnan(j):
                    all_values.append(j) # adds all non-NaN values to the list
                    
        temp_list = []   
        if len(all_values) > 18: # there are 21 total pixels, 19 or more are needed to have a decent representation and those will be added to the dataset
            temp_list.append(time)
            temp_list.append(mean_number)
            temp_list.append(var_type)

            dictionary_date_mean[teljari] = temp_list    #add time and mean value to the dictionary
            teljari += 1

        
    dataframe_df = pd.DataFrame.from_dict(dictionary_date_mean, orient='index', columns=['date', 'type', 'mean']) #convert dictionary to dataframe and output it
    dataframe_df['date'] = pd.to_datetime(dataframe_df['date'])
    return dataframe_df

            
df_tp = process_datasets(ds_resample_tp_clip_where, 'tp')
df_t2m = process_datasets(ds_resample_t2m_clip_where, 't2m')
print(len(df_tp), len(df_t2m))

236 670


In [7]:
#merge the two dataframes on the date
df_merged_era5 = pd.merge(df_tp, df_t2m, on='date', how='inner')
# df_merged['Date'] = pd.to_datetime(df_merged['Date'])
print(df_merged_era5.to_markdown())

|     | date                |   type_x | mean_x   |   type_y | mean_y   |
|----:|:--------------------|---------:|:---------|---------:|:---------|
|   0 | 2017-01-27 00:00:00 |      0   | tp       |     -2.6 | t2m      |
|   1 | 2017-01-28 00:00:00 |      0.1 | tp       |     -3.6 | t2m      |
|   2 | 2017-02-03 00:00:00 |      0.3 | tp       |     -5.3 | t2m      |
|   3 | 2017-02-06 00:00:00 |      0.2 | tp       |     -6.7 | t2m      |
|   4 | 2017-02-07 00:00:00 |      0   | tp       |    -12.3 | t2m      |
|   5 | 2017-02-08 00:00:00 |      0   | tp       |    -13.7 | t2m      |
|   6 | 2017-02-09 00:00:00 |      0   | tp       |    -12.6 | t2m      |
|   7 | 2017-02-10 00:00:00 |      0.2 | tp       |    -14.2 | t2m      |
|   8 | 2017-02-11 00:00:00 |      0.1 | tp       |    -12.1 | t2m      |
|   9 | 2017-02-15 00:00:00 |      0.1 | tp       |     -3.8 | t2m      |
|  10 | 2017-02-21 00:00:00 |      0   | tp       |    -14.6 | t2m      |
|  11 | 2017-02-24 00:00:00 |      0.1

# ASF SLC stuff

In [8]:
def Point_in_polygon(x, y, polygon):
    """Check if a point (x, y) is inside a polygon defined by a list of coordinates.
    Parameters:
    - x, y: Coordinates of the point to be checked.
    - polygon: List of tuples representing the vertices of the polygon.
    xxx ChatGPT"""
    n = len(polygon)
    inside = False

    p1x, p1y = polygon[0]
    for i in range(n + 1):
        p2x, p2y = polygon[i % n]
        if y > min(p1y, p2y) and y <= max(p1y, p2y) and x <= max(p1x, p2x):
            if p1y != p2y:
                xinters = (y - p1y) * (p2x - p1x) / (p2y - p1y) + p1x
                if p1x == p2x or x <= xinters:
                    inside = not inside
        p1x, p1y = p2x, p2y
    return inside

In [9]:
#import the excel file with Vindefjallen corner coordinates and measurements.
csv_file_path_2023 = r"Data\Vindefjallen_data\Vindefjallen_cleaning\Vindefjallen_Granule_processing\vindefjallen_corner_coords_processing_2023_middletime.csv"

df_measurements = pd.read_csv(csv_file_path_2023) # Read the CSV file into a DataFrame
vindefjallen_triangleCorners_numpy_2023 = df_measurements.to_numpy()
df_measurements['date'] = pd.to_datetime(df_measurements['date']) #convert date to datetime for later function

In [10]:
aoi = 'POLYGON((15.5702 66.3423,15.4488 66.3354,15.4242 66.2903,15.346 66.2493,15.2874 66.2273,15.2277 66.2165,15.145 66.1704,15.1619 66.0775,15.2024 66.0619,15.3474 66.0313,15.5304 65.9813,15.6281 65.9312,15.63 65.8965,15.741 65.8534,15.8627 65.8514,16.0288 65.8348,16.1754 65.8159,16.2049 65.8012,16.339 65.7877,16.4512 65.8283,16.2466 65.864,16.121 65.9472,16.1321 66.0205,16.2592 66.0296,16.4784 65.9773,16.6739 65.9642,16.8497 65.9721,16.9572 66.0108,16.7932 66.0502,16.5999 66.0381,16.4575 66.0817,16.2843 66.1399,16.2622 66.1561,15.9537 66.251,15.5702 66.3423))'
opts_slc = {
    'platform': asf.PLATFORM.SENTINEL1,
    'beamMode': asf.BEAMMODE.IW,
    'start': '2017-01-01T01:00:00Z',
    'end': '2023-06-11T23:59:59Z',
    'processingLevel' : asf.PRODUCT_TYPE.SLC
}
asf_coords_slc = asf.geo_search(intersectsWith=aoi, **opts_slc)
print(f'{len(asf_coords_slc)} results found - SLC')

3528 results found - SLC


Create a new version of vindefjallen_triangleCorners_numpy_era5



In [11]:
'''
The dates from the measurements dataframe are offset by 1 day and then compared to the dates in the ERA5 dataframe.
If they match, a new row is created with the offset date and the values from the ERA5 dataframe are added to the new row.
'''
df_measurements_copy = df_measurements.copy()
cols = list(df_measurements_copy.columns)
new_df = pd.DataFrame(columns=cols)

for i in range(len(df_measurements_copy)):

    for j in range(len(df_merged_era5)):
        offset_pls1 = df_measurements_copy.loc[i, "date"] + pd.DateOffset(days=1) #offset plus 1 day
        offset_min1 = df_measurements_copy.loc[i, "date"] - pd.DateOffset(days=1) #offset minus 1 day
        
        if df_merged_era5.loc[j, "date"] == offset_pls1: #if the date in the merged dataframe is equal to the date in the measurements dataframe + 1 day
            # Create a new row from the measurements dataframe and append the new offset date to it. The add it to a brand new dataframe
            mrT = df_measurements_copy.loc[i].to_frame().T #convert the row from measurement df to a dataframe
            mrT.loc[i, "era5_new_date"] = offset_pls1.strftime("%Y-%m-%d") #add the new date to the new dataframe
            mrT.loc[i, "era5_plusminus"] = "plus"
            mrT.loc[i, "era5_precip"] = df_merged_era5.loc[j, "type_x"]
            mrT.loc[i, "era5_temp"] = df_merged_era5.loc[j, "type_y"]
            new_df = pd.concat([new_df, mrT], ignore_index=True, axis=0) #concatenate the new dataframe with the row dataframe

        if df_merged_era5.loc[j, 'date'] == offset_min1:
            # Create a new row from the measurements dataframe and append the new offset date to it. The add it to a brand new dataframe
            mrT = df_measurements_copy.loc[i].to_frame().T #convert the row from measurement df to a dataframe
            mrT.loc[i, "era5_new_date"] = offset_min1.strftime("%Y-%m-%d") #add the new date to the new dataframe
            mrT.loc[i, "era5_plusminus"] = "minus" #add the new date to the new dataframe
            mrT.loc[i, "era5_precip"] = df_merged_era5.loc[j, "type_x"]
            mrT.loc[i, "era5_temp"] = df_merged_era5.loc[j, "type_y"]
            new_df = pd.concat([new_df, mrT], ignore_index=True, axis=0) #concatenate the new dataframe with the row dataframe

new_df = new_df.sort_values(by=['era5_plusminus'], ascending=False)
new_df['date'] = new_df['date'].astype(str).str[:10]
print('fin')
print(len(new_df))
vindefjallen_triangleCorners_numpy_era5 = new_df.to_numpy()# Convert the DataFrame to a NumPy array for easier coding


fin
351


In [35]:
the_path = r"C:\Users\trygg\Documents\Master_project\Data\Vindefjallen_data\Vindefjallen_cleaning\Vindefjallen_Granule_processing\vindefjallen_corner_coords_processing_2023_middletime_era5.csv"
new_df.to_csv(the_path, index=True)

In [12]:
print(new_df.to_markdown())

|     |   ID |   OG_ID | Triangle   | Corner   | TriangleCorner   | date       | SnowDepth   |   SnowDepth_Noplus |   temperature |   x_4326 |   y_4326 |   x_3006 |      y_3006 | MiddleTime   | era5_new_date   | era5_plusminus   |   era5_precip |   era5_temp |
|----:|-----:|--------:|:-----------|:---------|:-----------------|:-----------|:------------|-------------------:|--------------:|---------:|---------:|---------:|------------:|:-------------|:----------------|:-----------------|--------------:|------------:|
| 117 |  154 |     162 | AC-35      | B        | AC-35B           | 2017-03-06 | 95          |                 95 |         -10   |  16.5544 |  66.0232 |   570478 | 7.32337e+06 | 12:47:30     | 2017-03-07      | plus             |           0.1 |       -14.9 |
| 152 |  196 |     204 | AC-27      | B        | AC-27B           | 2019-03-11 | 30          |                 30 |         -15   |  15.7733 |  66.1063 |   534949 | 7.33197e+06 | 10:15:00     | 2019-03-12      | plus 

In [13]:
##### ASF SLC dictionary
def points_in_granule(vindefjallen_triangleCorners_numpy, era5=False):
    ordabok_granule_id_slc = {}
    teljari = 0
    for asf_coord in asf_coords_slc:
        polygon = asf_coord.geometry['coordinates'][0]
        measurment_in_granule_ID = []
        measurment_in_granule_cornerTriangle = []
        measurment_in_granule_date = []
        measurment_in_granule_depth = []
        measurment_in_granule_temperature = []
        measurment_in_granule_era5_date = ''

        # for corner in vindefjallen_triangleCorners_numpy_celcius: #Old
        for corner in vindefjallen_triangleCorners_numpy:
            point = (corner[9], corner[10])  # x, y coordinates from the excel file
            if era5 == True:
                start_time_match = corner[14] == asf_coord.properties['startTime'][:10]
            elif era5 == False:
                start_time_match = corner[5] == asf_coord.properties['startTime'][:10]

            if Point_in_polygon(point[0], point[1], polygon) and start_time_match:
                measurment_in_granule_ID.append(corner[0])
                measurment_in_granule_date.append(corner[5])
                measurment_in_granule_depth.append(corner[6])
                measurment_in_granule_cornerTriangle.append(corner[4])
                measurment_in_granule_temperature.append(corner[8])
                if era5 == True:
                    measurment_in_granule_era5_date = corner[14]

        if measurment_in_granule_date:
            ordabok_granule_id_slc[teljari] = {
                'granule_ID': asf_coord.properties['sceneName'],
                'orbitProp': asf_coord.properties['flightDirection'],
                'pathNr': asf_coord.properties['pathNumber'],
                'frameNr': asf_coord.properties['frameNumber'],
                'granule_date': asf_coord.properties['startTime'][:10],
                'startTime': asf_coord.properties['startTime'],
                'measurement_date': measurment_in_granule_date[0],
                'measurement_id': measurment_in_granule_ID,
                'measurement_cornerTriangle': measurment_in_granule_cornerTriangle,
                'measurement_depth': measurment_in_granule_depth,
                'measurement_temperature': measurment_in_granule_temperature,
                'nr_of_measurements': len(measurment_in_granule_ID),
                'polygon': polygon,
                'era5_date': measurment_in_granule_era5_date
            }
            teljari += 1

    print('length:', len(ordabok_granule_id_slc))
    return ordabok_granule_id_slc

ordabok_granule_id_slc_era5 = points_in_granule(vindefjallen_triangleCorners_numpy_era5, era5 = True)
ordabok_granule_id_slc = points_in_granule(vindefjallen_triangleCorners_numpy_2023, era5 = False)

length: 31
length: 57


In [31]:
len(ordabok_granule_id_slc_era5)

31

In [32]:
# #export to csv and no index
# df_export = pd.DataFrame.from_dict(ordabok_granule_id_slc_era5, orient='index')

# df_export.to_csv(r"Data\Vindefjallen_data\Vindefjallen_cleaning\Vindefjallen_Granule_processing\list_of_slc_granules_2023_ERA5.csv", index=False)

In [ ]:
for i in ordabok_granule_id_slc_era5:
    print()
    print(ordabok_granule_id_slc_era5[i]['granule_date'], ordabok_granule_id_slc_era5[i]['granule_ID'][17:21] + '-' + ordabok_granule_id_slc_era5[i]['granule_ID'][21:23] + '-' + ordabok_granule_id_slc_era5[i]['granule_ID'][23:25]+ ' | ' + ordabok_granule_id_slc_era5[i]['granule_ID'][26:28] + ':' + ordabok_granule_id_slc_era5[i]['granule_ID'][28:30]
          , ordabok_granule_id_slc_era5[i]['granule_ID'][-4:])

In [14]:
#Þessi kóði er til þess að búa til lista yfir hverja staka mælingu sem hefur granule
#Updated for 2023, go to git for old code

# ordabok_granule_id = ordabok_granule_id_slc
def granule_every_point_dict_func(ordabok_granule_id, vindefjallen_triangleCorners_numpy_2023, era5 = False):
    counter = 0
    granule_every_point_dict = {}
    for i in ordabok_granule_id:
        for j in range(len(ordabok_granule_id[i]['measurement_cornerTriangle'])):
            for k in vindefjallen_triangleCorners_numpy_2023:
                if era5 == True:
                    if ordabok_granule_id[i]['measurement_cornerTriangle'][j] == k[4] and ordabok_granule_id[i]['era5_date'] == k[14]:
                        granule_every_point_dict[counter] = {
                            'granule_id': ordabok_granule_id[i]['granule_ID'],
                            'granule_date': ordabok_granule_id[i]['granule_date'],
                            'measurement_date': ordabok_granule_id[i]['measurement_date'],
                            'measurement_id': ordabok_granule_id[i]['measurement_id'][j],
                            'snow_depth': ordabok_granule_id[i]['measurement_depth'][j],
                            'temperature': ordabok_granule_id[i]['measurement_temperature'][j],
                            'corner': ordabok_granule_id[i]['measurement_cornerTriangle'][j],
                            'x_coord': k[9],
                            'y_coord': k[10],
                            'x_coord3006': k[11],
                            'y_coord3006': k[12],
                            'era5_date': ordabok_granule_id[i]['era5_date']
                        }
                        counter += 1
                elif era5 == False:
                    if ordabok_granule_id[i]['measurement_cornerTriangle'][j] == k[4] and ordabok_granule_id[i]['measurement_date'] == k[5]:
                        granule_every_point_dict[counter] = {
                            'granule_id': ordabok_granule_id[i]['granule_ID'],
                            'granule_date': ordabok_granule_id[i]['granule_date'],
                            'measurement_date': ordabok_granule_id[i]['measurement_date'],
                            'measurement_id': ordabok_granule_id[i]['measurement_id'][j],
                            'snow_depth': ordabok_granule_id[i]['measurement_depth'][j],
                            'temperature': ordabok_granule_id[i]['measurement_temperature'][j],
                            'corner': ordabok_granule_id[i]['measurement_cornerTriangle'][j],
                            'x_coord': k[9],
                            'y_coord': k[10],
                            'x_coord3006': k[11],
                            'y_coord3006': k[12],
                        }
                        counter += 1
                        
    granule_df = pd.DataFrame.from_dict(granule_every_point_dict, orient='index')

    print('fin', len(granule_df))
    return granule_df

slc_granule_every_point_dict_df_era5 = granule_every_point_dict_func(ordabok_granule_id_slc_era5, vindefjallen_triangleCorners_numpy_era5, era5 = True)

slc_granule_every_point_dict_df = granule_every_point_dict_func(ordabok_granule_id_slc, vindefjallen_triangleCorners_numpy_2023, era5 = False)

fin 234
fin 398


In [15]:
'''
This cell is used to find the measurements that are in both the era5 and non-era5 datasets.
Its then possible to compare the measurements and see if they are the same.
'''
list_non_era = list(slc_granule_every_point_dict_df['measurement_id'].unique())
list_era5 = list(slc_granule_every_point_dict_df_era5['measurement_id'].unique())
list_non_era.sort()
list_era5.sort()

new_df_IDs = pd.DataFrame(columns=['id','non_ERA', 'Era5'])
list_x = list(range(0, 499))
new_df_IDs['id'] = list_x

# add the measurement id from era5 and non era5 datasets
for i in range(len(df_measurements)):
    for j in list_non_era:
        if new_df_IDs.loc[i, 'id'] == j:
            new_df_IDs.loc[i, 'non_ERA'] = j
    for k in list_era5:
        if new_df_IDs.loc[i, 'id'] == k:
            new_df_IDs.loc[i, 'Era5'] = k
print(new_df_IDs.to_markdown())

|     |   id |   non_ERA |   Era5 |
|----:|-----:|----------:|-------:|
|   0 |    0 |       nan |    nan |
|   1 |    1 |         1 |    nan |
|   2 |    2 |         2 |    nan |
|   3 |    3 |         3 |    nan |
|   4 |    4 |         4 |      4 |
|   5 |    5 |         5 |    nan |
|   6 |    6 |         6 |    nan |
|   7 |    7 |         7 |      7 |
|   8 |    8 |       nan |      8 |
|   9 |    9 |       nan |      9 |
|  10 |   10 |        10 |     10 |
|  11 |   11 |        11 |     11 |
|  12 |   12 |        12 |     12 |
|  13 |   13 |       nan |    nan |
|  14 |   14 |       nan |    nan |
|  15 |   15 |       nan |    nan |
|  16 |   16 |        16 |    nan |
|  17 |   17 |        17 |    nan |
|  18 |   18 |       nan |    nan |
|  19 |   19 |        19 |    nan |
|  20 |   20 |        20 |    nan |
|  21 |   21 |        21 |    nan |
|  22 |   22 |        22 |    nan |
|  23 |   23 |        23 |    nan |
|  24 |   24 |        24 |    nan |
|  25 |   25 |        25 |  

In [29]:
slc_granule_every_point_dict_df_era5
slc_granule_every_point_dict_df_era5[slc_granule_every_point_dict_df_era5['measurement_id']==30]

,granule_id,granule_date,measurement_date,measurement_id,snow_depth,temperature,corner,x_coord,y_coord,x_coord3006,y_coord3006,era5_date
169,S1A_IW_SLC__1SDV_20180330T052131_20180330T0521...,2018-03-30,2018-03-29,30,85,-10.0,AC-42A,15.760981,65.908407,534661.4522,7309911.059,2018-03-30
177,S1A_IW_SLC__1SDV_20180328T053741_20180328T0538...,2018-03-28,2018-03-29,30,85,-10.0,AC-42A,15.760981,65.908407,534661.4522,7309911.059,2018-03-28


In [34]:
#stats for the new dataframe
## new column where non_ERA and Era5 are the same
new_df_IDs["match"] = new_df_IDs["non_ERA"] == new_df_IDs["Era5"]

## new column where era5 is non nan and non_ERA is nan
new_df_IDs["only_era5"] = new_df_IDs["Era5"].notnull() & new_df_IDs["non_ERA"].isnull()


# count the number of True values in the same column
print(new_df_IDs["match"].value_counts())
print('')
print(new_df_IDs["only_era5"].value_counts())
print('')
# how many non nan values are in the non_ERA column
print(new_df_IDs["non_ERA"].notnull().sum())
print(new_df_IDs["Era5"].notnull().sum())

match
False    428
True      71
Name: count, dtype: int64

only_era5
False    429
True      70
Name: count, dtype: int64

347
141
